imports - numpy just to read data

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

parameter 'patience' is the number of epochs to proceed without improvement

In [ ]:
early_stopping_monitor = EarlyStopping(patience=3)

read the data so we can find the number of nodes in the input layer (n_cols)

In [ ]:
data_file = 'hourly_wages.csv'

use pandas to examine the file since we had a problem loading into numpy

In [ ]:
df = pd.read_csv(data_file)

In [ ]:
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.columns

In [ ]:
df.plot(x='education_yrs', y='wage_per_hour',kind='scatter')

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html

In [ ]:
df.corr()

In [ ]:
df.describe()

first row was a text headers which numpy could not handle. 
had to add "skiprows=1" parameter to get the loadtxt method to work

In [ ]:
# predictors = np.loadtxt(data_file, delimiter=',', skiprows=1)

In [ ]:
df.info()

In [ ]:
df.union = df.union.astype('bool')
df.female = df.female.astype('bool')
df.marr = df.marr.astype('bool')
df.south = df.south.astype('bool')
df.manufacturing = df.manufacturing.astype('bool')
df.construction = df.construction.astype('bool')

In [ ]:
df.columns

In [ ]:
df = (df-df.mean())/df.std()

In [ ]:
predictors = df.drop(['union','education_yrs','experience_yrs',
                      'age', 'female', 'marr', 'south', 'manufacturing','construction',
                     ], axis=1)

In [ ]:
# predictors = df.drop(['wage_per_hour','union','education_yrs','experience_yrs',
#                       'age', 'female', 'marr', 'south', 'manufacturing','construction',
#                      ], axis=1)

In [ ]:
df.head()

based on correlation, use only education_yrs and age for features

In [ ]:
df.plot()

In [ ]:
target = df.wage_per_hour

In [ ]:
n_cols = predictors.shape[1]

In [ ]:
n_cols

Sequential model - each layer connected only to the previous layer

In [ ]:
from keras import optimizers

In [ ]:
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
model = Sequential()

build up model layer at a time - 'Dense' layers are fully connected

In [ ]:
model.add(Dense(100, activation='relu', input_shape = (n_cols,)))

In [ ]:
model.add(Dense(100, activation='relu'))

In [ ]:
model.add(Dense(1))

dump out the model configuration

In [ ]:
model.get_config()

'adam' is a good, general purpose optimizer that adjusts the learning rate as it goes

In [ ]:
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
# model.compile(optimizer=adam, loss='mean_squared_error')

In [ ]:
# model.compile(optimizer='adam', loss='mean_squared_error')

need to split out the target column from the data

In [ ]:
model.fit(predictors, target, validation_split=0.3, epochs=20)

In [ ]:
model.fit(predictors, target, validation_split=0.3, epochs=20, 
          callbacks = [early_stopping_monitor])

this model is not doing very well. Maybe need to look at scaling the data. 
Find the mean of each feature and divide by standard deviation.

In [ ]:
from keras.models import load_model

In [ ]:
model.save('model_file.h5')

In [ ]:
my_model = load_model('model_file.h5')